In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(r'./countries/countries.db')
def rsq(sql):
    return pd.read_sql(sql, conn)

In [3]:
rsq("""-- Select country code for countries in the Middle East
SELECT code
FROM countries
WHERE region = 'Middle East';""").head()

,code
0,ARE
1,ARM
2,AZE
3,BHR
4,GEO


In [5]:
rsq("""-- Select unique language names
SELECT DISTINCT name
FROM languages
-- Order by the name of the language
ORDER BY name;""").shape

(396, 1)

In [7]:
rsq("""SELECT DISTINCT name
FROM languages
-- Add syntax to use bracketed subquery below as a filter
WHERE code IN 
    (SELECT code
    FROM countries
    WHERE region = 'Middle East')
ORDER BY name;""").shape

(27, 1)

In [9]:
rsq("""-- Select code and name of countries from Oceania
SELECT code, name
FROM countries
WHERE continent = 'Oceania'
ORDER BY name ASC;""")

,code,name
0,ASM,American Samoa
1,AUS,Australia
2,FJI,Fiji Islands
3,PYF,French Polynesia
4,GUM,Guam
5,KIR,Kiribati
6,MHL,Marshall Islands
7,FSM,"Micronesia, Federated States of"
8,NRU,Nauru
9,NCL,New Caledonia


In [8]:
rsq("""SELECT code, name
FROM countries
WHERE continent = 'Oceania'
-- Filter for countries not included in the bracketed subquery
  AND code NOT IN 
    (SELECT code
    FROM currencies);""")

,code,name
0,ASM,American Samoa
1,FJI,Fiji Islands
2,GUM,Guam
3,FSM,"Micronesia, Federated States of"
4,MNP,Northern Mariana Islands


In [11]:
rsq("""-- Select average life_expectancy from the populations table
SELECT AVG(life_expectancy) 
FROM populations 
WHERE year = 2015;
-- Filter for the year 2015
-- ___""")

,AVG(life_expectancy)
0,71.676342


In [10]:
rsq("""SELECT *
FROM populations
-- Filter for only those populations where life expectancy is 1.15 times higher than average
WHERE life_expectancy > 1.15 * (SELECT AVG(life_expectancy)
                               FROM populations
                               WHERE year = 2015)
  AND year = 2015;""")

,pop_id,country_code,year,fertility_rate,life_expectancy,size
0,21,AUS,2015,1.833,82.451220,23789752.0
1,376,CHE,2015,1.540,83.197561,8281430.0
2,356,ESP,2015,1.320,83.380488,46443994.0
3,134,FRA,2015,2.010,82.670732,66538391.0
4,170,HKG,2015,1.195,84.278049,7305700.0
5,174,ISL,2015,1.930,82.860976,330815.0
6,190,ITA,2015,1.370,83.490244,60730582.0
7,194,JPN,2015,1.460,83.843659,126958472.0
8,340,SGP,2015,1.240,82.595122,5535002.0
9,374,SWE,2015,1.880,82.551220,9799186.0


In [13]:
rsq("""-- Select relevant fields from cities table
SELECT name, country_code, urbanarea_pop
FROM cities
-- Filter using a subquery on the countries table
WHERE name IN
  (SELECT capital
   FROM countries)
ORDER BY urbanarea_pop DESC;""").shape

(66, 3)

In [14]:
rsq("""-- Find top nine countries with the most cities
SELECT countries.name AS country, COUNT(*) AS cities_num
FROM countries
LEFT JOIN cities
ON countries.code = cities.country_code
GROUP BY country
-- Order by count of cities as cities_num
ORDER BY cities_num DESC, country
LIMIT 9;""")

,country,cities_num
0,China,36
1,India,18
2,Japan,11
3,Brazil,10
4,Pakistan,9
5,United States,9
6,Indonesia,7
7,Russian Federation,7
8,South Korea,7


In [15]:
rsq("""SELECT countries.name AS country,
-- Subquery that provides the count of cities   
  (SELECT COUNT(*)
   FROM cities
   WHERE cities.country_code = countries.code) AS cities_num
FROM countries
ORDER BY cities_num DESC, country
LIMIT 9;""")

,country,cities_num
0,China,36
1,India,18
2,Japan,11
3,Brazil,10
4,Pakistan,9
5,United States,9
6,Indonesia,7
7,Russian Federation,7
8,South Korea,7


In [17]:
rsq("""-- Select code, and language count as lang_num
SELECT code, COUNT(*) AS lang_num
FROM languages
GROUP BY code;""")

,code,lang_num
0,ABW,7
1,AFG,4
2,AGO,12
3,AIA,1
4,ALB,4
...,...,...
207,WSM,2
208,YEM,1
209,ZAF,13
210,ZMB,19


In [16]:
rsq("""-- Select local_name and lang_num from appropriate tables
SELECT local_name, sub.lang_num
FROM countries,
    (SELECT code, COUNT(*) AS lang_num
     FROM languages
     GROUP BY code) AS sub
-- Where codes match    
WHERE countries.code = sub.code
ORDER BY lang_num DESC;""")

,local_name,lang_num
0,Zambia,19
1,YeItyop´iya,16
2,Zimbabwe,16
3,Bharat/India,14
4,Nepal,14
...,...,...
193,Saint Kitts and Nevis,1
194,Deutschland,1
195,San Marino,1
196,Al-´Arabiya as-Sa´udiya,1


In [18]:
rsq("""-- Select relevant fields
-- Select country code, inflation_rate, and unemployment_rate from economies.
-- Filter code for the set of countries which do not contain the words "Republic" or "Monarchy" in their gov_form.
-- ___
-- WHERE year = 2015 
--   AND code NOT IN
-- Subquery returning country codes filtered on gov_form
-- Select relevant fields
SELECT code, inflation_rate, unemployment_rate
FROM economies
WHERE year = 2015 
  AND code NOT IN
-- Subquery returning country codes filtered on gov_form
    (SELECT code
     FROM countries
     WHERE (gov_form LIKE '%Monarchy%' OR gov_form LIKE '%Republic%'))
ORDER BY inflation_rate;""")

,code,inflation_rate,unemployment_rate
0,AFG,-1.549,NaN
1,CHE,-1.140,3.178
2,PRI,-0.751,12.000
3,ROU,-0.596,6.812
4,TLS,0.553,NaN
5,MNE,1.204,NaN
6,SRB,1.392,18.200
7,HKG,3.037,3.296
8,ARE,4.070,NaN
9,MAC,4.564,1.825


In [19]:
rsq("""-- From cities, select the city name, country code, proper population, and metro area population, as well as the field city_perc, which calculates the proper population as a percentage of metro area population for each city (using the formula provided).
-- Filter city name with a subquery that selects capital cities from countries in 'Europe' or continents with 'America' at the end of their name.
-- Exclude NULL values in metroarea_pop.
-- Order by city_perc (descending) and return only the first 10 rows.
-- Select fields from cities
SELECT 
	name, 
    country_code, 
    city_proper_pop, 
    metroarea_pop,
    city_proper_pop / metroarea_pop * 100 AS city_perc
FROM cities
-- Use subquery to filter city name
WHERE name IN
  (SELECT capital
   FROM countries
   WHERE (continent = 'Europe'
   OR continent LIKE '%America'))
-- Add filter condition such that metroarea_pop does not have null values
	  AND metroarea_pop IS NOT NULL
-- Sort and limit the result
ORDER BY city_perc DESC
LIMIT 10;""")

,name,country_code,city_proper_pop,metroarea_pop,city_perc
0,Lima,PER,8852000,10750000.0,82.344186
1,Bogota,COL,7878783,9800000.0,80.395745
2,Moscow,RUS,12197596,16170000.0,75.433494
3,Vienna,AUT,1863881,2600000.0,71.687731
4,Montevideo,URY,1305082,1947604.0,67.009618
5,Caracas,VEN,1943901,2923959.0,66.481815
6,Rome,ITA,2877215,4353775.0,66.085523
7,Brasilia,BRA,2556149,3919864.0,65.210145
8,London,GBR,8673713,13879757.0,62.491822
9,Budapest,HUN,1759407,2927944.0,60.090186
